In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData()

In [4]:
first_np, second_np = utils.process_splitted(first_np, second_np)

linearized complete!


In [5]:
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [6]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 130 pretrained vectors found.


In [7]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [9]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/phase2_tmp/test')

In [10]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


In [11]:
# ====================
# set parameters here
# ====================

title = 'inverted_noGudu3'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 200
hidden_size = 100
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.2

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=pretrained_word2vec
freeze_embedding=False,

In [12]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!


/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


OCR_rnn(
  (emb): Embedding(130, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [13]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.269516   | 89.510288  | 0.211826 | 91.74  | 115.69
   2    |   0.167491   | 93.930041  | 0.149220 | 94.67  | 117.17
   3    |   0.135115   | 94.868313  | 0.113680 | 96.15  | 116.18
   4    |   0.110586   | 95.855967  | 0.117874 | 96.15  | 117.88
   5    |   0.099598   | 96.341564  | 0.089371 | 96.78  | 116.68
   6    |   0.091146   | 96.576132  | 0.095895 | 96.52  | 116.23
   7    |   0.084081   | 96.868313  | 0.087554 | 96.81  | 116.54
   8    |   0.081602   | 96.954733  | 0.079905 | 97.22  | 116.39
   9    |   0.078095   | 97.065844  | 0.087839 | 96.85  | 117.89
  10    |   0.073250   | 97.390947  | 0.080517 | 97.15  | 116.38
  11    |   0.068934   | 97.497942  | 0.075685 | 97.22  | 116.66
  12    |   0.071013   | 97.530864  | 0.075126 | 97.00  | 116.72
  13    |   0.065013   | 97.547325  | 0.090661 | 96.7

In [14]:
mu.saveModel(title, model)

In [15]:
model = mu.getModel(title)
print(model)

OCR_rnn(
  (emb): Embedding(130, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [16]:
loss, acc = tester.test(test_dataloader=test_dataloader,
                        device=device,
                        model=model,
                        title=title)

test loss:  0.10629417292075231
test acc:  97.33333333333334
saved precision and recall results to file!


In [17]:
with open('../result/final', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [18]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!
